### ДИПЛОМНАЯ РАБОТА

#### ТЕМА: модели генерации текста на примере генератора хайку

#### ПЛАН РАБОТЫ

- Character-based model
- Word-based model
- GAN / Sense-vector? Autoencoders?

#### ВВЕДЕНИЕ

здесь будет текст 

In [8]:
# prepare data

file_path_src = "haiku"
with open(file_path_src, encoding="utf-8") as f:
    lines = f.readlines()
    by3lines = []
    allHaiku = []
    for line in lines:
        if line == "\n":
            allHaiku.append(by3lines)
            by3lines = []
        else:
            by3lines.append(line.lower())
    # print(allHaiku);

In [9]:
def to_oneline(data):
    return ["".join(row) for row in data]

максимальная длина стихотворения

In [16]:
hh = to_oneline(allHaiku)

In [17]:
max(map(lambda s: len(s), hh))

84

In [12]:
from __future__ import print_function
from keras.callbacks import LambdaCallback
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file
import numpy as np
import random
import sys
import io

/Users/petriko/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [18]:
text = "".join(hh)

In [19]:
chars = sorted(list(set(text)))
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

total chars: 45


нарезаем исходный текст на сэмплы: кусочки длины maxlen (входные данные), а следующий символ целевая переменная.

In [22]:
maxlen = 10
step = 2
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('nb sequences:', len(sentences))

nb sequences: 11929


In [23]:
sentences[:10]

['луна-прово',
 'на-проводн',
 '-проводник',
 'роводник \n',
 'водник \nзо',
 'дник \nзове',
 'ик \nзовет:',
 ' \nзовет: "',
 'зовет: "за',
 'вет: "загл']

векторизируем строки

In [24]:
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

построим простую LSTM сеть

In [25]:
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars), activation='softmax'))

optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

Instructions for updating:
Colocations handled automatically by placer.


In [26]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [27]:
def on_epoch_end(epoch, _):
    # Function invoked at end of each epoch. Prints generated text.
    print()
    print('----- Generating text after Epoch: %d' % epoch)

    start_index = random.randint(0, len(text) - maxlen - 1)
    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print('----- diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + maxlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(400):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

In [28]:
print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

In [29]:
model.fit(x, y,
          batch_size=128,
          epochs=100,
          callbacks=[print_callback])

Instructions for updating:
Use tf.cast instead.
Epoch 1/100
11929/11929 [==============================] - 2s 158us/step - loss: 3.0038

----- Generating text after Epoch: 0
----- diversity: 0.2
----- Generating with seed: "расстегива"
расстегива ветеростолистолилестолилилилижи поливетоли веголодоролетололетолилика порисетоловетолодосетолисеролилилетодоголидолосеролетоливолежелолосе полетоли велеголеролололиси полетолистолоси ветолосеетолилилетолилет весе полетелилилигоколиза постоликолетололитоли поливетололоветоласетолисе полетолилолилиси вези полостолетолисто полетололистиликолисти ветолилити вереза полелилети полетили полилилоликолили
----- diversity: 0.5
----- Generating with seed: "расстегива"
расстегивавеза ветовее долоредостовветелый пелесеродетеримимесили вестос слевирисожеделиголисаколелулени.
е дасеведи 
вегете дестомотер зажере подоста подореголаседоповет веполилизилилекобеслолеленесяу мочежодо лепом полетека ворев секо дакомализижетолиреликапос дегожопо аберы долете илилет

строй дим. 
таких мнею дожде пола, ка
Epoch 9/100
11929/11929 [==============================] - 1s 116us/step - loss: 1.2784

----- Generating text after Epoch: 8
----- diversity: 0.2
----- Generating with seed: "н открыл
п"
н открыл
по замыл ветер!
в ветной свогром -
далы в преде.
и о дом и 
ворном свочер 
сод и поднеи 
сядит мастини
на перази мол.
листой горот.
высалые всень рассволи. 
но станил состаль
и полнок усила.
сталый рестинние дождьм
я в чере на светер. 
прия за вод 
ворох за прется 
престил сворит 
ворох светен,
я в преднуй ветер!
в ветной свогром -
дало всег ветер! 
пойно вистой воду 
воротся вишнив 
вишни хождим
и
----- diversity: 0.5
----- Generating with seed: "н открыл
п"
н открыл
паом за воле
воробе палод расоскател свол.
весеннее пери.
друг угорот.
лист поразимая ветвь

светит мо ночки запил окрастин 
и пресное свили.
сельные свелит своче 
следут окрасти.
на скаруг свотев 
а присла гнядий дождь.
здом и вышу 
горсть диже ть, цветок 
в салуну стора в нече
меестом своз

олько прохла том ноче - трувит. 
небачел птоветел ид и 
перевот мне скуди!
чаль прузуй ветки. и все могаселькай домица исенный дерев -
ид радит этой руси пличают всень снег на фудев
им цветушке
вылочной деме -
и том рассвые рассветко. -
и там касоля, как красинает мустом -
слаго вышина 
соль встол нобака!
в салуннею крубки лига в наше - 
рвстипа луная. 
тук гумина лишь поймёе 
респелась раск зам
луг сторка 
----- diversity: 0.5
----- Generating with seed: "олько прох"
олько прохла покад.
ходо пелент. 
слепек поднул сон, 
претеннуе утра. 
присный поли, 
дрестил гнабо.
о вдались раскруги верну, 
вотки рассвые рассветко. гнаша льшаний прис.
о ветка вотразат.
земеей поддым цветко.
о ветка вошка в цесяний систво 
светуст калы. 
ветва вышил постел свотро лестья назан...
зимней вечер!
я на замной росенки!
четва все же белтало поднелись раскойка криго перетьям
погдихо перу
а пол
----- diversity: 1.0
----- Generating with seed: "олько прох"
олько прохлодний делев.
холожда дод!
рудика ветром
в…с

кродямный реке.
я вышу, простинокамоматень
так помоцальнока заморы, 
в черень задночи но черень порой снегнуга 
шляди бесь
лавые леском бодяли - 
грысли на 
взрогн пе едушки о прохлтута содний крекит.
западной деревокак явшах прама, не клочин. вдриг
----- diversity: 0.5
----- Generating with seed: "ам, где эх"
ам, где эхо оржелт вод -
как придовали -
ивых в поле.
друзачелковедной .
дума светкихск
вел вся горд,
дрезля догди.
пола в слувай светеро, 
траст своз-
а на воздан. 
тылия са мата, какля,
прослался на цветых 
крубит кошлиника.
остинний огле том 
снет на прабоетке егляшну!
бадько остьни покля.
я на проснотьсахагли на ари гу.
отушисы лестьяна проздим, подались и погла и 
праснулся,
пушна все зневыт подуму бро
----- diversity: 1.0
----- Generating with seed: "ам, где эх"
ам, где эхо однелет. 
баки!и бажам блядуги оли
и дожет звезвутрак плов диже:теплы. полном
ился мню, это олне ногот стонь - и кумый ко ветка!
в ридзет обуда 
сень в птиле. 
сиры рапри иглазу радан
окля и заннои
игная

ветер ист прики. 
силина родит. 
плетый луна.
задерений
----- diversity: 0.5
----- Generating with seed: "ождя.
о не"
ождя.
о небод ресть может оно, кркоду молит ветер! ветра все заятот.
замые леский. 
холтий осьной кроб.
к чама сомой!
в предник на жигь 
пездой в везни 
все веннует солмотовала -
растаск бланого полетел. 
масень коподний
длядяям всехакасалкатся готдико ромитался нночай поломен!
лишьи па, 
ародит выпутся
снуга бледко. 
слены как роспот 
в этом самол содвель веэеруши цветит.
лестопрый вижей прабе.кей облатит.
----- diversity: 1.0
----- Generating with seed: "ождя.
о не"
ождя.
о небуда все:
наеседа тастарень нотничь
ь все дождум всех на борох.
обунула мает.
ветер здод 
ветрал свеюя ондалы топдалась па, полененье
погистом кака, дляди, поглевит. 
начелька жеро ходнокзвичной
жирни ! под рас левый лесть.
маселька – тено стролье
десеники в руки. 
восенний полете.
одино поивой
сопу студ -
пруноветря умазля водот -
и дышитик 
горим семний.
в тревит сон, 
отса на камот.
прамнался 

...

мой ветняю берог...
зимной домом...
но тиры полкой но зебяца.
я и -удни га сточкрик вида в 
ивет свошк верки, чикак.
забула раза.
о на запот.
поаз и преберко. 
холоколо восх вирные жкрыв,
в гразд
----- diversity: 1.2
----- Generating with seed: "и.
из умыв"
и.
из умывак наданус"ы в предний претдям.
малень пощенен!
холотов, но моте, 
в травы холоко, красумол.
– на капошодце.
неа рамойся замыет воловы 
в замот за ное
даже тыполо окарза.
неть, мотром но первоме
делего выщейка.
воты в паду.
дрихинные руст. 
перном вся замеемо
в него за одли.
о не прилет. 
мой нбе пу мутвет.
старны хотлици на иигну умыи
на привещ сабома 
слевет снечат, 
пресет незомолит.
прощиха ут
Epoch 66/100
11929/11929 [==============================] - 1s 109us/step - loss: 0.1392

----- Generating text after Epoch: 65
----- diversity: 0.2
----- Generating with seed: "д в небеса"
д в небеса вышку!
на светким шипли!
на рукой поле.
ник крастиц на свечер. 
прушина всенные меги.
со замыл оскдут осенний диховши.
остинный ок

смотрит на цветы 
брездует шлездил сия мое, кромои. 
соловые лоскралишь, 
госький не прсялуу на блед. 
преляст м нобеке.
домоданимий
проснуль кабамы
нагут, педя гля в берод на предеркий другутром
цветуть, как рым цветог. 
в шив лечну. суноо...
капи порглаши за очле
дань мост заколька в набушкой.теноют пустом!

бень п!рошано водоется
ворот рикавы.
я сой цветкой
состый родних:
ночте его пруглу
друга осемкит 
Epoch 74/100
11929/11929 [==============================] - 1s 115us/step - loss: 0.1364

----- Generating text after Epoch: 73
----- diversity: 0.2
----- Generating with seed: "лась и спи"
лась и спи.
верьновея в бежнуг в черево, пруговет!
заперет на саруки слявы.
удало вмол
напок в ветером посмнею снеги.
ед моловышел.
осенний вечер.
в певеровик виши
вих гругл 
ствыльсс светит мо моей,
ощень па но 
сверет днячи
- 
светрипа раскац оклала 
стучит кастарного 
имней тучик и прилес. 
солонный зименце.
старый рок. 
клень пожень в ресну  лишья бливит на светьюм 
слывы  раскля на холжег мой

пужало птуюти обадьный вит.
вдруг са веруг 
ризлачое однилкк.
видрави, короба!
за претов 
светлы ле тачел.
в пладули бирашь 
жеснь это обудали,
весевае листьк в уты.
сручать на свечый ветни.
с лушкг… но замлем.
Epoch 82/100
11929/11929 [==============================] - 1s 105us/step - loss: 0.1164

----- Generating text after Epoch: 81
----- diversity: 0.2
----- Generating with seed: "яталась за"
яталась занолиел дождний 
вопре в визра вся пресе моет оты облика.
но стаму прица...
лисе пороздали погля но 
леве слушкиб – на грозе
мылет как спорет моняля луна.
луноки пуяь.
ла том пожд
и д содний короткей од соленной груд. 
пежену под 
я на просну. 
снег нежты.
горд, запод рикнуйся на крачуе раскойкракаюся, птевет.
паёт пой не краса!
за праздалоко 
крекины на блетом ноча 
вотре стыет засекит -
из-ти как
----- diversity: 0.5
----- Generating with seed: "яталась за"
яталась зацвели души сниау тана, илсный полном 
стром ка точкий свотрот на предвоза гресен. 
под микай веткрёи ры кразали шла 

11929/11929 [==============================] - 1s 109us/step - loss: 0.1176

----- Generating text after Epoch: 89
----- diversity: 0.2
----- Generating with seed: "хрь в пути"
хрь в пути. 
сак леве в шам рас…
саста в реске. 
гна подям 
светкит завилинь
верно скарики гни крувиско замнола 
снег нах него мени
, 
– не вененимье
долся луна 
прековал осенний диждь.
поужду можелоте.
к какий ко мол
в ноге -него славы раска!
ц пага в тере
и дорий рветы. 
внов рассвыт могенья в вотер -
дуг об летках ном снег сногиа -
но чиро славний деревья
мошень стуши пряздуо илветер, 
ворох, за вотером 
----- diversity: 0.5
----- Generating with seed: "хрь в пути"
хрь в пути. 
сак левное
сорная вуша. 
прияля на бусу
веровет в зимной
снеракастывал светень, 
гос, песнень паино 
светщей оспа
виро в перевода вышнум –я домга.
зимний окна мой воко сакровью 
заму ворозду лестой
в лышной замерзи. 
гисткай квсвые расвели.
ночек небаи зажего зливе, ире 
присст мне сень. 
мореть сверер содницы земеньи.
х лень порозла в

одходит осень. 
тучит -вила вовытной снеги.
здям дая вдох 
все замериканы цветенья.
вдруг небабековыет ветер! ветка снегу 
спрегнул беруг...
зимной синиц витре…
уа ется кушли
на пребедажится, луна.
луна под ном расули. 
сомне лестона истакени
лудажились
расперказали, лист д.
из икой двеум 
снет мой венной.
добужде у ростдет
в перезая в замос.
ихлу ид на ввдух не стувый мень
посокнул ость!
на ваноче по венну
----- diversity: 0.5
----- Generating with seed: "одходит ос"
одходит осень. 
тучит веснове.
холода уадли.
как пь 
светкнылая,и..
селнок. однаца топой труной сень!
нажер, переговых луни
сночки ком.
замяте луна, и пукел моженоком
ях тум ветва в неба
е ветка в неш е.
ихо омна лунног. 
день ветлям 
сетдаленьямая вдлуг !
ветером мойм.
в ветерня все са то сого дожи.
иходне уа реко ночки, крусткой толький делевий на берье.
какае пе преленоть в ноче.
волото о белдинены гор, 
----- diversity: 1.0
----- Generating with seed: "одходит ос"
одходит осень. 
тучит весна уги плаша
под я в тера...


смотрит на цветы 

зимняя луна 

и там и присну.

---

желтое ободкломомале

доже тиво подуеть.

зимый вечной

#### ИСПОЛЬЗОВАННЫЕ ИСТОЧНИКИ

http://lib.ru/JAPAN/BASE/base.txt_with-big-pictures.html

http://rozamira.org/lib/themes/vestniki/hokku.htm

http://lib.co.ua/poems/unknown/russkiehokku.jsp (начало)

http://haiku.ru/frog/classic.htm


Word-level RNN:

https://github.com/rdcolema/word-level-rnn-for-text-generation/blob/master/word_gen.py
https://gist.github.com/maxim5/c35ef2238ae708ccb0e55624e9e0252b

